In [1]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers
!pip install torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 17.3 MB/s eta 0:00:00
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.20.1
    Uninstalling graphviz-0.20.1:
      Successfully uninstalled graphviz-0.20.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 344.5/344.5 kB 5.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp310-cp310-linux_x86_64.whl size=661673 sha256=aa80805dffda808fb7128998bdb2d9d8f8744a1f09f1abc443e93ffbd9696ea8
  Stored in directory: /root/.cache/pip/wheels/1a/1e/0d/99f55911d90f2b95b9f7c176d5813ef3622894a4b30fde6bd3
Successfully built gluonnlp
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 39.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 72.9 MB/s eta 0:00:00
     ━━━━━

In [2]:
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-_bgwbbqi/kobert-tokenizer_6c4e67f806054a41bdcf91057582803c
  Running command git clone --filter=blob:none --quiet https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-_bgwbbqi/kobert-tokenizer_6c4e67f806054a41bdcf91057582803c
  Resolved https://github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done
  Created wheel for kobert_tokenizer: filename=kobert_tokenizer-0.1-py3-none-any.whl size=4632 sha256=842d4df45c541585c2268e43d689bf78099920a7ef0fb7d9418ca685e18e9f78
  Stored in directory: /tmp/pip-ephem-wheel-cache-mi5l5kxc/wheels/e9/1a/3f/a864970e8a169c176befa3c4a1e07aa612f69195907a4045fe
Successfully built kobert_tokenizer


In [3]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [4]:
# Hugging Face를 통한 모델 및 토크나이저 Import
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [5]:
# GPU 사용 시
device = torch.device("cuda:0")

In [6]:
# 필요한 클래스 정의

class BERTSentenceTransform:
    r"""BERT style data transformation.

    Parameters
    ----------
    tokenizer : BERTTokenizer.
        Tokenizer for the sentences.
    max_seq_length : int.
        Maximum sequence length of the sentences.
    pad : bool, default True
        Whether to pad the sentences to maximum length.
    pair : bool, default True
        Whether to transform sentences or sentence pairs.
    """

    def __init__(self, tokenizer, max_seq_length,vocab, pad=True, pair=True):
        self._tokenizer = tokenizer
        self._max_seq_length = max_seq_length
        self._pad = pad
        self._pair = pair
        self._vocab = vocab

    def __call__(self, line):
        """Perform transformation for sequence pairs or single sequences.

        The transformation is processed in the following steps:
        - tokenize the input sequences
        - insert [CLS], [SEP] as necessary
        - generate type ids to indicate whether a token belongs to the first
        sequence or the second sequence.
        - generate valid length

        For sequence pairs, the input is a tuple of 2 strings:
        text_a, text_b.

        Inputs:
            text_a: 'is this jacksonville ?'
            text_b: 'no it is not'
        Tokenization:
            text_a: 'is this jack ##son ##ville ?'
            text_b: 'no it is not .'
        Processed:
            tokens: '[CLS] is this jack ##son ##ville ? [SEP] no it is not . [SEP]'
            type_ids: 0     0  0    0    0     0       0 0     1  1  1  1   1 1
            valid_length: 14

        For single sequences, the input is a tuple of single string:
        text_a.

        Inputs:
            text_a: 'the dog is hairy .'
        Tokenization:
            text_a: 'the dog is hairy .'
        Processed:
            text_a: '[CLS] the dog is hairy . [SEP]'
            type_ids: 0     0   0   0  0     0 0
            valid_length: 7

        Parameters
        ----------
        line: tuple of str
            Input strings. For sequence pairs, the input is a tuple of 2 strings:
            (text_a, text_b). For single sequences, the input is a tuple of single
            string: (text_a,).

        Returns
        -------
        np.array: input token ids in 'int32', shape (batch_size, seq_length)
        np.array: valid length in 'int32', shape (batch_size,)
        np.array: input token type ids in 'int32', shape (batch_size, seq_length)

        """

        # convert to unicode
        text_a = line[0]
        if self._pair:
            assert len(line) == 2
            text_b = line[1]

        tokens_a = self._tokenizer.tokenize(text_a)
        tokens_b = None

        if self._pair:
            tokens_b = self._tokenizer(text_b)

        if tokens_b:
            # Modifies `tokens_a` and `tokens_b` in place so that the total
            # length is less than the specified length.
            # Account for [CLS], [SEP], [SEP] with "- 3"
            self._truncate_seq_pair(tokens_a, tokens_b,
                                    self._max_seq_length - 3)
        else:
            # Account for [CLS] and [SEP] with "- 2"
            if len(tokens_a) > self._max_seq_length - 2:
                tokens_a = tokens_a[0:(self._max_seq_length - 2)]

        # The embedding vectors for `type=0` and `type=1` were learned during
        # pre-training and are added to the wordpiece embedding vector
        # (and position vector). This is not *strictly* necessary since
        # the [SEP] token unambiguously separates the sequences, but it makes
        # it easier for the model to learn the concept of sequences.

        # For classification tasks, the first vector (corresponding to [CLS]) is
        # used as as the "sentence vector". Note that this only makes sense because
        # the entire model is fine-tuned.
        #vocab = self._tokenizer.vocab
        vocab = self._vocab
        tokens = []
        tokens.append(vocab.cls_token)
        tokens.extend(tokens_a)
        tokens.append(vocab.sep_token)
        segment_ids = [0] * len(tokens)

        if tokens_b:
            tokens.extend(tokens_b)
            tokens.append(vocab.sep_token)
            segment_ids.extend([1] * (len(tokens) - len(segment_ids)))

        input_ids = self._tokenizer.convert_tokens_to_ids(tokens)

        # The valid length of sentences. Only real  tokens are attended to.
        valid_length = len(input_ids)

        if self._pad:
            # Zero-pad up to the sequence length.
            padding_length = self._max_seq_length - valid_length
            # use padding tokens for the rest
            input_ids.extend([vocab[vocab.padding_token]] * padding_length)
            segment_ids.extend([0] * padding_length)

        return np.array(input_ids, dtype='int32'), np.array(valid_length, dtype='int32'),\
            np.array(segment_ids, dtype='int32')


In [7]:
import pandas as pd

# 엑셀 파일 불러오기
df1 = pd.read_excel('/content/감성대화말뭉치(최종데이터)_Training.xlsx')
df2 = pd.read_excel('/content/감성대화말뭉치(최종데이터)_Validation.xlsx')
df3 = pd.read_excel('/content/한국어_단발성_대화_데이터셋.xlsx')

# 필요없는 열 삭제 (문장과 감정만 남김)
df1 = df1[['사람문장1', '감정_대분류']]
df2 = df2[['사람문장1', '감정_대분류']]
df3 = df3[['Sentence', 'Emotion']]

# 열 이름 변경 (Sentence와 Emotion으로 통일)
df1.rename(columns={'사람문장1': 'Sentence', '감정_대분류': 'Emotion'}, inplace=True)
df2.rename(columns={'사람문장1': 'Sentence', '감정_대분류': 'Emotion'}, inplace=True)

# 데이터프레임 합치기 (감성대화말뭉치)
combined_df = pd.concat([df1, df2], ignore_index=True)

# 'Emotion' 열에서 '상처'를 '슬픔'으로 변경 (감성대화말뭉치)
combined_df['Emotion'] = combined_df['Emotion'].replace('상처', '슬픔')

# 변경된 데이터프레임 확인
combined_df[combined_df['Emotion'] == '슬픔']

# 'Emotion' 열에서 '혐오'를 '분노'로 변경 (한국어_단발성_대화_데이터셋)
df3['Emotion'] = df3['Emotion'].replace('혐오', '분노')

# 변경된 데이터프레임 확인
df3[df3['Emotion'] == '분노']

# 'Emotion' 열에서 '중립'을 '평온'으로 변경 (한국어_단발성_대화_데이터셋)
df3['Emotion'] = df3['Emotion'].replace('중립', '평온')

# 변경된 데이터프레임 확인
df3[df3['Emotion'] == '평온']

# 'Emotion' 열에서 '공포'를 '불안'으로 변경 (한국어_단발성_대화_데이터셋)
df3['Emotion'] = df3['Emotion'].replace('공포', '불안')

# 'Emotion' 열에서 '놀람'을 '당황'으로 변경 (한국어_단발성_대화_데이터셋)
df3['Emotion'] = df3['Emotion'].replace('놀람', '당황')

# 'Emotion' 열에서 '행복'을 '기쁨'으로 변경 (한국어_단발성_대화_데이터셋)
df3['Emotion'] = df3['Emotion'].replace('행복', '기쁨')
# 데이터프레임 합치기 (한국어_단발성_대화_데이터셋 + 감성대화말뭉치)
final_df = pd.concat([combined_df, df3], ignore_index=True)
df = final_df


<ipython-input-7-089a05abcafe>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1.rename(columns={'사람문장1': 'Sentence', '감정_대분류': 'Emotion'}, inplace=True)
<ipython-input-7-089a05abcafe>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2.rename(columns={'사람문장1': 'Sentence', '감정_대분류': 'Emotion'}, inplace=True)


In [21]:
print(df.columns)



Index(['Sentence', 'Emotion'], dtype='object')


In [8]:
df.loc[(df['Emotion'] == "불안"), 'Emotion'] = 0  #불안 => 0
df.loc[(df['Emotion'] == "당황"), 'Emotion'] = 1  #당황 => 1
df.loc[(df['Emotion'] == "분노"), 'Emotion'] = 2  #분노 => 2
df.loc[(df['Emotion'] == "슬픔"), 'Emotion'] = 3  #슬픔 => 3
df.loc[(df['Emotion'] == "평온"), 'Emotion'] = 4  #평온 => 4
df.loc[(df['Emotion'] == "기쁨"), 'Emotion'] = 5  #기쁨 => 5
df.loc[(df['Emotion'] == "분노"), 'Emotion'] = 6  #분노 => 6

In [9]:
data_list = []
for q, label in zip(df['Sentence'], df['Emotion'])  :
    data = []
    data.append(q)
    data.append(str(label))

    data_list.append(data)

In [10]:
#데이터분리
from sklearn.model_selection import train_test_split
dataset_train, dataset_test = train_test_split(data_list, test_size=0.25, random_state=0)

print(len(dataset_train))
print(len(dataset_test))

72648
24217


In [11]:
from gluonnlp.data import TSVDataset

dataset_train = pd.DataFrame(dataset_train, columns=['Sentence', 'Emotion'])
dataset_test = pd.DataFrame(dataset_test, columns=['Sentence', 'Emotion'])

dataset_train.to_csv('dataset_train.tsv', sep='\t', index=False)
dataset_test.to_csv('dataset_test.tsv', sep='\t', index=False)

In [30]:
import gluonnlp as nlp

# 데이터셋 로드
dataset_train = nlp.data.TSVDataset('dataset_train.tsv', num_discard_samples=1)
dataset_test = nlp.data.TSVDataset('dataset_test.tsv',  num_discard_samples=1)

# 데이터셋 확인
sample_train = dataset_train[0]
sample_test = dataset_test[0]
print("Number of Fields:", len(sample))  # 데이터셋의 필드 개수 출력

print("Sample from dataset_train:", sample_train)
print("Sample from dataset_test:", sample_test)



Number of Fields: 2
Sample from dataset_train: [' 답장 더 이상 안하겠대요.', '3']
Sample from dataset_test: ['휴가를 맞아 친구들과 바다로 낚시를 하러 갔다가 폭풍우를 만나서 나만 살아남아서 죄책감이 들고 슬퍼.', '3']


In [13]:
# Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 1
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [34]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, vocab, max_len,
                 pad, pair):
        transform = BERTSentenceTransform(bert_tokenizer, max_seq_length=max_len,vocab=vocab, pad=pad, pair=pair)
        #transform = nlp.data.BERTSentenceTransform(
        #    tokenizer, max_seq_length=max_len, pad=pad, pair=pair)
        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel = BertModel.from_pretrained('skt/kobert-base-v1', return_dict=False)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file, padding_token='[PAD]')

#
data_train = BERTDataset(dataset_train, 0, 1, tokenizer, vocab, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tokenizer, vocab, max_len, True, False)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


IndexError: ignored

In [ ]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=7,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate

        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)

    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)

        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [ ]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [ ]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [ ]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [ ]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

In [ ]:
model1 = model # 원본 모델 복사

In [ ]:
def predict(predict_sentence): # 모델을 통한 예측
  data = [predict_sentence, '0']
  dataset_another = [data]

  another_test = BERTDataset(dataset_another, 0, 1, tokenizer, vocab, max_len, True, False)
  test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)

  model1.eval()

  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
          token_ids = token_ids.long().to(device)
          segment_ids = segment_ids.long().to(device)
          valid_length= valid_length
          label = label.long().to(device)
          out = model1(token_ids, valid_length, segment_ids)

          result = ""

          for i in out:

            # 소프트맥스 함수를 적용하여 확률값 계산
            probs = torch.nn.functional.softmax(out, dim=-1)
            probs = probs.detach().cpu().numpy()

            # 가장 높은 확률을 갖는 클래스 선택
            predicted_class = np.argmax(probs, axis=1)[0]
            # 확률 계산
            probability = probs[0, predicted_class]*100


            class_names = ["불안", "당황", "분노", "슬픔", "평온", "기쁨", "분노"]

            # 결과 및 확률 출력
            result = f">> 입력하신 내용은 {probability:.2f}% 확률로 {class_names[predicted_class]}으로 예측됩니다."
            print(result)

In [ ]:
#질문 무한 반복 1을 누르면 종료
while True:
  sentence = input("하고싶은 말을 입력해주세요 : ")
  if sentence == "1":
    break
  print(sentence)
  predict(sentence)
  print("\n")
